In [ ]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
import numpy as np
from math import atan, atan2, sin, cos
from random import uniform

In [ ]:
eps = np.finfo(float).eps
def cross(A, B):
    return A[0]*B[1] - A[1]*B[0]

def potential(A, B, P):
    C =  np.dot(A, B) - np.dot(A, P) +np.dot(B, P) - np.dot(B, B)
    D = -np.dot(A, B) - np.dot(A, P) +np.dot(B, P) + np.dot(A, A)
    E = cross(A, B) - cross(A, P) + cross(B, P)
    if np.abs(E) < eps:
        return 0.0
    else:
        return (atan(C/E) - atan(D/E)) / E

def winding(A, B, P):
    C = cross(A-P, B-P)
    D = np.dot(A-P, B-P)
    return atan2(C,D)/2.0/np.pi

def cost(L,P):
    m, n, o = L.shape
    p = 0.0
    w = 0.0
    for j in range(m):
        for i in range(n):
            p += potential(L[j,i-1,:], L[j,i,:], P)
            w += winding(L[j,i-1,:], L[j,i,:], P)
    w = np.abs(w)
    if np.abs(p) < eps:
        return 0.0
    elif w < 0.5:
        return np.sqrt(n/-p)
    else:
        return -np.sqrt(n/-p)
    
def polys(m, n, r1=2.0, r2=1.0):
    L = np.zeros((m, n, 2))
    for j in range(m):
        if m==1:
            c = np.zeros(2)
        else:
            c = np.array([cos(2*np.pi*j/m*uniform(0.8,1.0))*r1*uniform(0.5,1.5), sin(2*np.pi*j/m*uniform(0.8,1.0))*r1*uniform(0.5,1.5)])
        for i in range(n):
            L[j,i,:] = c + np.array([cos(2*np.pi*i/n*uniform(0.8,1.0))*r2*uniform(0.5,1.5), sin(2*np.pi*i/n*uniform(0.8,1.0))*r2*uniform(0.5,1.5)])
    return L

In [ ]:
L=polys(2, 4, 2.0, 1.0)

In [ ]:
x = np.linspace(-5, 5, 50)
y = np.linspace(-5, 5, 50)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = cost(L, np.array([X[i, j],Y[i, j]]))

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
m, n, o = L.shape
idx = np.linspace(0,n,n+1, dtype=int) % n
for j in range(m):
    ax.plot(L[j,idx,0], L[j,idx,1], np.zeros(n+1), linewidth=2, color='k')

ax.contour(X, Y, Z, 10, alpha=0.8)
ax.plot_surface(X, Y, Z, cmap=matplotlib.cm.jet, linewidth=0.25, antialiased=False, alpha=0.2, edgecolor=[0,0,0,0.05])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Cost')
plt.show()